In [1]:
import pandas as pd
import geopandas as gpd
import os
import requests
import multiprocessing as mp
import itertools


In [5]:
def call_census_table(state_list, table_name, key):
    
    result_df = pd.DataFrame()
    
    # querying at census tract level
    for state in state_list:
        if table_name.startswith('DP'):
            address = f'https://api.census.gov/data/2020/acs/acs5/profile?get=NAME,{table_name}&for=tract:*&in=state:{state}&in=county:*'
        elif table_name.startswith('S'):
            address = f'https://api.census.gov/data/2020/acs/acs5/subject?get=NAME,{table_name}&for=tract:*&in=state:{state}&in=county:*'
        elif table_name.startswith('B'):
            address = f'https://api.census.gov/data/2020/acs/acs5?get=NAME,{table_name}&for=tract:*&in=state:{state}&in=county:*'
        else:
            raise AttributeError('Proper Table Name Is Needed.')
            
        # print(state, table_name)    
        response = requests.get(f'{address}&key={key}').json()
        result_ = pd.DataFrame(response)
        
        result_.columns = response[0]
        result_.drop(0, axis=0, inplace=True)
        
        result_df = pd.concat([result_, result_df]).reset_index(drop=True)
        
    # result_df = result_df.rename(columns={'GEO_ID':'GEOID_T'})
    result_df['GEOID_T'] = result_df.apply(lambda x: x['state'] + x['county'] + x['tract'], axis=1)
    result_df[table_name] = result_df[table_name].astype(float)
        
    return result_df[['GEOID_T', table_name]]


def census_data_retrieval(attr, state_list, tract, census_dic, API_Key):
    attr_df = pd.DataFrame({'GEOID_T':tract['GEOID'].unique().tolist()})

    print(attr)
    cols = list(census_dic.keys())
    cols.append('GEOID_T')

    if type(census_dic[attr]) == str:
        temp_table = call_census_table(state_list, census_dic[attr], API_Key)
        attr_df = attr_df.merge(temp_table, on='GEOID_T')
        attr_df = attr_df.rename(columns={census_dic[attr]: attr})
    else:
        for table in census_dic[attr][0]: # Retrieve numerator variables
            temp_table = call_census_table(state_list, table, API_Key)
            attr_df = attr_df.merge(temp_table, on='GEOID_T')

        temp_table = call_census_table(state_list, census_dic[attr][1], API_Key) # Retrieve denominator variable
        attr_df = attr_df.merge(temp_table, on='GEOID_T')

        # Calculate the ratio of each variable
        attr_df[attr] = attr_df[census_dic[attr][0]].sum(axis=1) / attr_df[census_dic[attr][1]] * 100

    # Remove intermediate columns used for SVI related census calculation
    attr_df = attr_df[attr_df.columns.intersection(cols)]

    # attr_df.to_csv(os.path.join(data_dir, 'census_geometry', f'census_data_{attr}.csv'))

    return attr_df

In [3]:
census_dic = {
                "EP_POV150" : [['S1701_C01_040E'], 'S1701_C01_001E'],
                "EP_UNEMP"  : 'DP03_0009PE',
                "EP_HBURD"  : [['S2503_C01_028E', 'S2503_C01_032E', 'S2503_C01_036E', 'S2503_C01_040E'], 
                            'S2503_C01_001E'],
                "EP_NOHSDP" : 'S0601_C01_033E',
                "EP_UNINSUR" : 'S2701_C05_001E',
                "EP_AGE65" : 'S0101_C02_030E',
                "EP_AGE17" : [['B09001_001E'], 
                            'S0601_C01_001E'],
                "EP_DISABL" : 'DP02_0072PE',
                "EP_SNGPNT" : [['B11012_010E', 'B11012_015E'], 'DP02_0001E'],
                "EP_LIMENG" : [['B16005_007E', 'B16005_008E', 'B16005_012E', 'B16005_013E', 'B16005_017E', 'B16005_018E', 
                                'B16005_022E', 'B16005_023E', 'B16005_029E', 'B16005_030E', 'B16005_034E', 'B16005_035E',
                                'B16005_039E', 'B16005_040E', 'B16005_044E', 'B16005_045E'], 
                            'B16005_001E'],
                "EP_MINRTY" : [['DP05_0071E', 'DP05_0078E', 'DP05_0079E', 'DP05_0080E', 
                                'DP05_0081E', 'DP05_0082E', 'DP05_0083E'],
                            'S0601_C01_001E'],
                "EP_MUNIT" : [['DP04_0012E', 'DP04_0013E'], 
                            'DP04_0001E'],
                "EP_MOBILE" : 'DP04_0014PE',
                "EP_CROWD" : [['DP04_0078E', 'DP04_0079E'], 
                            'DP04_0002E'],
                "EP_NOVEH" : 'DP04_0058PE',
                "EP_GROUPQ": [['B26001_001E'], 
                            'S0601_C01_001E'],
}

API_Key = '5ad4c26135eaa6a049525767607eecd39e19d237' # Census api key

data_dir = os.getcwd()
# data_dir = '/anvil/projects/x-cis220065/x-cybergis/compute/Aging_Dams'

tract = gpd.read_file(os.path.join(data_dir, 'census_geometry', 'census_tract_from_api.geojson'))

state_lookup = pd.read_csv(os.path.join(data_dir, 'census_geometry', 'state_lookup.csv'))
state_lookup = state_lookup.loc[state_lookup['ContiguousUS'] == 1]
state_lookup['FIPS'] = state_lookup['FIPS'].astype(str)
state_list = list(state_lookup.apply(lambda x:x['FIPS'] if len(x['FIPS']) == 2 else '0' + x['FIPS'], axis=1))
state_list = state_list[0:3]

In [6]:
a1 = census_data_retrieval('EP_POV150', state_list, tract, census_dic, API_Key)
a2 = census_data_retrieval('EP_UNEMP', state_list, tract, census_dic, API_Key)
a3 = census_data_retrieval('EP_HBURD', state_list, tract, census_dic, API_Key)

EP_POV150
EP_UNEMP
EP_HBURD


In [14]:
census_df = pd.DataFrame({'GEOID_T':tract['GEOID'].unique().tolist()})
census_df = census_df.loc[census_df.apply(lambda x: True if x['GEOID_T'][0:2] in state_list else False, axis=1)]
census_df = census_df.reset_index(drop=True)
census_df
# census_df.loc[census_df['GEOID_T'].startswith()]

,GEOID_T
0,04013104207
1,04013216822
2,04017940011
3,04013422655
4,04013103400
...,...
4019,01095030301
4020,04013614900
4021,05051012001
4022,05125010406


In [15]:
census_df = pd.DataFrame({'GEOID_T':tract['GEOID'].unique().tolist()})
census_df = census_df.loc[census_df.apply(lambda x: True if x['GEOID_T'][0:2] in state_list else False, axis=1)]
census_df = census_df.reset_index(drop=True)

for result in [a1, a2, a3]:
    census_df = census_df.merge(result, on='GEOID_T')
    
census_df

,GEOID_T,EP_POV150,EP_UNEMP,EP_HBURD
0,04013104207,17.133183,1.8,27.188690
1,04013216822,4.939809,3.0,13.735741
2,04017940011,54.516472,14.6,18.266254
3,04013422655,13.548770,0.5,22.793488
4,04013103400,10.065448,2.0,23.749313
...,...,...,...,...
4019,01095030301,6.685018,3.3,11.342352
4020,04013614900,7.934600,2.3,28.483492
4021,05051012001,26.348692,6.2,28.688131
4022,05125010406,11.853696,2.9,9.184727


In [38]:
census_df = pd.read_csv('/Users/jparkgeo/Git_Repo/population_vulnerable_to_dam_failure/census_geometry/census_data.csv')
census_df['GEOID_T'] = census_df['GEOID_T'].apply(lambda x:"{:011d}".format(x))
census_df

,GEOID_T,EP_POV150,EP_UNEMP,EP_HBURD,EP_NOHSDP,EP_UNINSUR,EP_AGE65,EP_AGE17,EP_DISABL,EP_SNGPNT,EP_LIMENG,EP_MINRTY,EP_MUNIT,EP_MOBILE,EP_CROWD,EP_NOVEH,EP_GROUPQ
0,06029002001,78.559791,16.8,72.377286,38.1,5.7,15.5,45.356794,23.2,21.366699,7.745933,90.583252,17.122040,0.0,4.234841,38.7,4.952753
1,48071710500,35.863200,5.7,21.739130,32.6,16.5,17.8,23.716934,18.1,12.379227,11.454280,46.529505,3.933949,22.4,3.079710,2.1,3.481800
2,55025011505,21.948843,0.4,30.560690,11.2,5.0,13.6,24.431138,8.3,7.886630,1.704261,34.562874,14.037193,0.0,2.402957,3.5,1.676647
3,06059086701,23.473556,3.7,32.570659,25.9,7.0,15.4,20.326819,13.2,2.557201,20.062334,77.619521,12.361937,14.3,18.797667,4.1,0.518936
4,48201341203,12.720665,4.1,29.839572,7.7,14.4,14.9,10.643821,18.6,4.919786,4.994629,43.250260,49.873631,0.0,6.631016,3.3,0.259605
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83504,12005001302,24.707872,5.2,38.408190,7.0,10.2,18.1,22.263223,26.8,8.685601,1.049352,23.570111,6.977378,20.2,1.651255,5.6,0.307503
83505,04013811700,14.461467,3.9,27.132227,13.2,9.6,17.5,14.763231,17.4,2.461362,1.546267,33.890436,0.000000,13.7,1.202061,3.2,0.324977
83506,12105012511,19.953326,0.0,14.135626,37.9,13.8,20.4,21.586931,20.4,0.095511,13.041725,90.665111,0.000000,2.3,3.342884,0.0,0.000000
83507,47035970602,17.206266,2.0,16.656857,8.5,7.6,26.6,16.187990,19.2,6.415539,0.000000,3.133159,0.361944,16.0,2.001177,2.3,0.000000
